#### Imports

In [1]:
from scipy import spatial, sparse
from scipy.stats import chi2
from collections import Counter
from num2words import num2words
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.externals import joblib 
from nltk.tokenize import word_tokenize
from sklearn.model_selection import KFold
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import os
import imp
import gzip
import copy
import nltk
import pickle
import scipy
import string
import gensim
import operator
import datetime

import numpy as np
import pandas as pd
import LDA_ELJST as lda
import matplotlib.pyplot as plt

In [2]:
import utils as my_utils
from sentiment import SentimentAnalysis

### Read Data

In [3]:
dataset_name = "amazon_musical"

In [4]:
dataset = pd.read_pickle("datasets/datadf_amazon_musical")

In [5]:
dataset.head(2)

,reviewerID,asin,reviewerName,helpful,unixReviewTime,reviewText,sentiment,reviewTime,summary,cleaned,text
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]",1393545600,"Not much to write about here, but it does exac...",5.0,"02 28, 2014",good,"[much, write, doe, exactly, supposed, filter, ...",much write doe exactly supposed filter pop sou...
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",1363392000,The product does exactly as it should and is q...,5.0,"03 16, 2013",Jake,"[product, doe, exactly, quite, affordable, rea...",product doe exactly quite affordable realized ...


In [6]:
count_matrix, _, vocabulary, words = my_utils.processReviews(dataset['text'].values)

In [7]:
Counter(dataset['sentiment'])

Counter({1.0: 214, 2.0: 245, 3.0: 756, 4.0: 2050, 5.0: 6785})

In [8]:
barren = np.where(count_matrix.sum(1)==0)[0]

In [9]:
barren

array([], dtype=int64)

In [10]:
ratings = dataset['sentiment'].values

### Making Edge_Dict

In [11]:
dataset_name

'amazon_musical'

In [12]:
pickle_in = open("resources/edges_amazon_musical_glove_nontrained_0.55.pickle","rb")
docs_edges = pickle.load(pickle_in)

In [13]:
docs_edges = np.delete(docs_edges, barren).tolist()

In [14]:
len(docs_edges)

10050

In [15]:
edge_dict__ = []
for doc in docs_edges:
    edge_dict_ = {}
    for i, j in doc:
        try:
            edge_dict_[i] += [j]
        except:
            edge_dict_[i] = [j]
        try:
            edge_dict_[j] += [i]
        except:
            edge_dict_[j] = [i]
    edge_dict__.append(edge_dict_)

In [16]:
len(edge_dict__[0])

7

In [17]:
from sklearn.metrics import silhouette_score, davies_bouldin_score

In [18]:
folder = "dumps/2019-09-27 17:06:47.184403_amazon_musical/"

# Model Evaluations

In [27]:
dumps = ['sampler_amazon_musical_n_topics_40',
         'sampler_amazon_musical_n_topics_60',
         'sampler_amazon_musical_n_topics_100']

In [28]:
def get_scores(file_):
    sampler = joblib.load(folder + file_)
    top_words = [i[1] for i in sampler.getTopKWords(5, words).items()]
    dt_distribution = sampler.theta()
    
    print my_utils.coherence_score(count_matrix, top_words, vocabulary), "\t", silhouette_score(count_matrix, dt_distribution.argmax(axis=1)), "\t", davies_bouldin_score(count_matrix, dt_distribution.argmax(axis=1))

In [29]:
for i in dumps:
    get_scores(i)

-29.227191104253784 	-0.09295563714187419 	13.246166769160407
-29.60745447381482 	-0.07945596386194118 	11.892525931879895
-30.17279215856641 	-0.0848287429385537 	10.338700921518077
